# Recorte da Imagem para depois Equalizar novamente
* Iremos recortar a imagem para depois equalizar novamente pois melhora a qualidade da equalização;
* Eliminados os espaços em preto desnecessários da imagem, ficando apenas com o recorte da mama;
* Assim, melhora significativamente a qualidade da equalização, e como consequencia, melhora a qualidade do constraste da imagem permitindo visualizar muito mais as estruturas da mama.

In [3]:
import pydicom as dicom
import cv2
import imutils

In [ ]:
localImg = "/mnt/d/ImagensDatasetFredrik/"
img = dicom.dcmread(localImg + "00342_20990909_L_CC_3.dcm")

ret, temp1 = cv2.threshold(img.pixel_array,127,255,cv2.THRESH_BINARY)
temp1=temp1.astype('uint8')

cnts = cv2.findContours(temp1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key= cv2.contourArea, reverse = True) #pegando as coordenadas que resultam na maior áre
rect = cv2.boundingRect(cnts[0])

x,y,w,h = rect
croped = (im.pixel_array)[y:y+h, x:x+w].copy() 

plt.imshow(exposure.equalize_adapthist(croped), cmap='gray')
plt.axis('off')
plt.savefig('/home/bernardo/Mestrado/fredrik-dataset/data-processed/'+filename_mlo['anon_filename'][i][0:20]+'.png')